互联网中充斥着大量钓鱼欺诈类网站。这类非法网站通常试图掩人耳目、充当正规网站，而实际上却是在窃取用户的身份、密码、交易等重要信息 机器学习在信息安全领域中的一个重要应用就是用来识别这些钓鱼网站。fraudulent.csv在data文件夹中。

fraudulent.csv文件中含有10,086条数据，每条数据含有18个特征以及1个标签。各个特征的含义如下：

contain_IP：网址中是否包涵ip，比如http://121.99.3.123/fake.html 包含ip。1表示包含，0表示不包含；
is_long：网址字符是否过长。1表示网址过长，0表示网址不长；
is_tinyurl：网址是否是短网址。比如https://bit.ly/2kXX6jV 就是短网址。1表示是短网址，0表示不是；
contain_at：网址是否包含“@”符号。1表示包含，0表示不包含；
contain_double_slash：网址是否包含“//”符号，该符号用来表示网址跳转。1表示包含，0表示不包含；
contain_dash：网址是否包含“-”符号，该符号经常帮助用来伪装真网站，比如www.my-taobao.com 。 1表示包含，0表示不包含；
contain_subdomain：网址是否包含子域名，比如www.ecnu.edu.cn 就包含edu和cn子域名。1表示包含，0表示不包含；
is_SSL：网址是否是https安全链接。1表示包含，0表示不包含；
with_long_history：网址所属的主域名存在的时间。1表示长久，0表示不长久；
contain_icon：网址网页是否有小图标。1表示包含，0表示不包含；
contain_ext_domain：该网页是否加载其他域名下的附件或者网页。1表示包含，0表示不包含；
contain_email_to：该网页是否包含发送邮件的组件。1表示包含，0表示不包含；
allow_right_click：该网页是否允许用户进行右击操作。1表示允许，0表示不允许；
contain_pop_up_windowL：该网页是否包含弹窗。1表示包含，0表示不包含；
contain_Iframe：该网页是否包含Iframe（嵌套网页）。1表示包含，0表示不包含；
has_DNSRecord：网址是否有DNS记录。1表示有，0表示无；
traffic：该网站的流量大小。1表示大，0表示小；
google_rank：该网址在google搜索中的排名。1表示高于同类网站的平均排名，0表示低于同类网站的平均排名；

y：表示网站是否是钓鱼欺诈网站，1表示是，0表示不是。
原始数据中含有大量缺失值，请自行处理这些缺失值（可以剔除缺失值过多的列或者使用众数填充等方法）。
将原始数据分为训练集、测试集（随机种子请设置为1）（若有需要可以将训练集进一步分为训练集和验证集）。

现在请建立一个二分类模型，使用训练集训练模型，再使用测试集测试模型。

评估指标为F1值

分类模型可采用：k-近邻、决策树、逻辑回归、支持向量机等。

可以与周围同学比较一下F1值的大小（越接近1越好），看看谁的数据预处理和分类模型更强。

In [ ]:
# 因为在做实验之前，不知道集中分类模型哪个的效果最好，所以我每个模型都做了，然后输出了咩个模型的F1值，这样最后再得出一个结论，比较客观
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, classification_report
def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

def handle_missing_values(data, threshold=0.5):
    # 计算每一列的缺失比例
    missing_ratio = data.isnull().mean()
    
    # 找出缺失比例超过阈值的列并删除
    columns_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
    if columns_to_drop:
        data = data.drop(columns=columns_to_drop)
    
    # 使用众数填充剩余的缺失值
    mode_values = data.mode().iloc[0]
    data_filled = data.fillna(mode_values)
    
    return data_filled

# 特征与标签分离
def separate_features_labels(data, label_column='y'):
    feature_cols = data.columns.drop(label_column)
    X = data[feature_cols]
    y = data[label_column]
    return X, y
# 数据集划分
def split_data(X, y, test_size=0.2, random_state=1):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    return X_train, X_test, y_train, y_test

# 模型训练
def train_models(X_train, y_train):
    models = {}
    
    # 逻辑回归
    lr = LogisticRegression(max_iter=1000, random_state=1)
    lr.fit(X_train, y_train)
    models['Logistic Regression'] = lr
    
    # 决策树
    dt = DecisionTreeClassifier(random_state=1)
    dt.fit(X_train, y_train)
    models['Decision Tree'] = dt
    
    # K近邻
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    models['K-Nearest Neighbors'] = knn
    
    # 支持向量机
    svm = SVC(kernel='linear', random_state=1)
    svm.fit(X_train, y_train)
    models['Support Vector Machine'] = svm
    
    return models

# 模型评估
def evaluate_models(models, X_test, y_test):
    f1_scores = {}
    
    for name, model in models.items():
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred)
        f1_scores[name] = f1
        print(f"\n{name}的F1值: {f1:.4f}")
    
    return f1_scores

def main():
    data_file = r'C:\Users\大头\Desktop\fraudulent.csv'
    
    data = load_data(data_file)
    
    data_clean = handle_missing_values(data, threshold=0.5)
    
    X, y = separate_features_labels(data_clean, label_column='y')
    
    X_train, X_test, y_train, y_test = split_data(X, y, test_size=0.2, random_state=1)
    
    models = train_models(X_train, y_train)
    
    f1_scores = evaluate_models(models, X_test, y_test)
    
    print("\n各模型的F1值比较:")
    for name, score in f1_scores.items():
        print(f"{name}: {score:.4f}")
    
    # 找出F1值最高的模型
    best_model = max(f1_scores, key=f1_scores.get)
    print(f"\n表现最佳的模型是: {best_model}，F1值为: {f1_scores[best_model]:.4f}")

# 执行主函数
if __name__ == "__main__":
    main()




Logistic Regression的F1值: 0.8510

Decision Tree的F1值: 0.8688

K-Nearest Neighbors的F1值: 0.8299

Support Vector Machine的F1值: 0.8558

各模型的F1值比较:
Logistic Regression: 0.8510
Decision Tree: 0.8688
K-Nearest Neighbors: 0.8299
Support Vector Machine: 0.8558

表现最佳的模型是: Decision Tree，F1值为: 0.8688
